Experiment1: what if instead of V = Wk(x), we use V = x

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-11-23 05:32:24--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2023-11-23 05:32:24 (26.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
!mkdir checkpoints

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import time
import os

In [ ]:
torch.manual_seed(170310087)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'{device = }')

device = 'cuda'


# PREPARING THE DATA

In [ ]:
with open('input.txt', 'r') as f:
    text = f.read()
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print('vocab:', ''.join(vocab))
print(f'{len(vocab)=}')

vocab: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
len(vocab)=65


In [ ]:
int_map = {ch:i for i,ch in enumerate(vocab)}
char_map = {i:ch for i,ch in enumerate(vocab)}

def encode(s):
    return [int_map[c] for c in s]

def decode(ii):
    return ''.join([char_map[i] for i in ii])


In [ ]:
class TinyShakespeare(torch.utils.data.Dataset):
    def __init__(self, data, cont_width):
        self.data = data
        self.cont_width = cont_width

    def __len__(self):
        return len(self.data) - self.cont_width

    def __getitem__(self, idx):
        x = self.data[idx: idx+self.cont_width].to(device)
        y = self.data[idx+1: idx+self.cont_width+1].to(device)
        return (x, y)

In [ ]:
data = torch.tensor(encode(text))
cont_width = 32
train_split = 0.9


# MODEL

In [ ]:
class SelfAttentionHead(nn.Module):
    def __init__(self, embd_size, head_size, cont_width):
        super().__init__()
        self.embd_size = embd_size # C
        self.head_size = head_size # H
        self.Wk = nn.Linear(embd_size, head_size, bias=True) # (C,H)
        self.Wq = nn.Linear(embd_size, head_size, bias=True) # (C,H)
        self.Wv = nn.Linear(embd_size, head_size, bias=True) # (C,H)
        self.register_buffer('mask', torch.tril(torch.ones(cont_width, cont_width))) # (T,T)

    def forward(self, x):
        # x is (B,T,C)
        _, T, _ = x.shape # we are extracting the shape because T may differ during generation
        Q = self.Wq(x) # (B,T,H)
        K = self.Wk(x) # (B,T,H)
        V = self.Wv(x) # (B,T,H)

        att = Q @ K.transpose(-2,-1) / self.head_size # (B,T,T)
        att = att.masked_fill(self.mask[:T, :T]==0, float('-inf')) # for decoder head only
        att = att.softmax(dim=2) # softmax for each query over all keys

        out = att @ V # (B,T,H)
        return out


class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embd_size, cont_width, n_heads):
        super().__init__()
        head_size = embd_size // n_heads
        self.heads = nn.ModuleList([SelfAttentionHead(embd_size, head_size, cont_width) for _ in range(n_heads)])
        self.linear = nn.Linear(embd_size, embd_size)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=2)  # [4, 8, 0]
        out = self.linear(out) # (B,T,C)
        return out


class Block(nn.Module):
    def __init__(self, embd_size, cont_width, n_heads):
        super().__init__()
        self.lay_norm1 = nn.LayerNorm(embd_size)
        self.multihead = MultiHeadSelfAttention(embd_size, cont_width, n_heads)
        self.dropout1 = nn.Dropout(0.1)
        self.lay_norm2 = nn.LayerNorm(embd_size)
        self.feedforward = nn.Sequential(nn.Linear(embd_size, 4*embd_size),
                                         nn.ReLU(),
                                         nn.Linear(4*embd_size, embd_size))
        self.dropout2 = nn.Dropout(0.1)

    def forward(self, x):
        x = x + self.dropout1(self.multihead(self.lay_norm1(x)))
        x = x + self.dropout2(self.feedforward(self.lay_norm2(x)))
        return x



class DecoderModel(nn.Module):

    def __init__(self, vocab_size, cont_width, embd_size, n_heads, n_blocks):
        super().__init__()
        self.cont_width = cont_width
        self.emb_table = nn.Embedding(vocab_size, embd_size)
        self.pos_table = nn.Embedding(cont_width, embd_size)
        self.dropout = nn.Dropout(0.1)
        self.blocks = nn.Sequential(*[Block(embd_size, cont_width, n_heads) for _ in range(n_blocks)])
        self.lay_norm = nn.LayerNorm(embd_size)
        self.lin = nn.Linear(embd_size, vocab_size)

    def forward(self, x):
        B, T = x.shape
        x = self.emb_table(x) + self.pos_table(torch.arange(T, device=device)) # (B,T,C) + (T,C)
        # x = self.dropout(x) # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.lay_norm(x) # (B,T,C)
        x = self.lin(x) # (B,T,vocab)
        return x

    def generate(self, x, max_tokens):
        for _ in range(max_tokens):
            inp = x[:, -cont_width:] # (B,T)
            out = self(inp) # (B,T,vocab)
            probs = out[:,-1,:].softmax(dim=-1) # (B,vocab)
            pred = torch.multinomial(probs, num_samples=1) # (B, )
            x = torch.cat((x, pred), dim=1)
        return x



In [ ]:
train_set = TinyShakespeare(data[:int(len(data) * train_split)], cont_width)
val_set = TinyShakespeare(data[int(len(data) * train_split):], cont_width)
print(f'{len(train_set)=}')
print(f'{len(val_set)=}')

len(train_set)=1003822
len(val_set)=111508


In [ ]:
embd_size = 128
n_heads = 8
n_blocks = 6
eval_batches = 5
batch_size = 8192

@torch.no_grad()
def evaluate(model, eval_batches):
    trainloader = torch.utils.data.DataLoader(train_set, batch_size=2048, shuffle=True)
    valloader = torch.utils.data.DataLoader(val_set, batch_size=2048, shuffle=False)
    criterion = nn.CrossEntropyLoss()
    loss = [0, 0]
    for split, dataloader in enumerate([trainloader, valloader]):
        for i, (x, y) in enumerate(trainloader):
            if i == eval_batches:
                break
            logits = model(x)
            loss[split] += criterion(logits.transpose(1, 2), y).item()

    return loss[0]/eval_batches, loss[1]/eval_batches

trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
# valloader = torch.utils.data.DataLoader(val_set, batch_size=4, shuffle=False)
model = DecoderModel(vocab_size, cont_width, embd_size, n_heads, n_blocks).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.005)
model.train()
start = time.time()

for epoch in range(5):
    for i, (x, y) in enumerate(trainloader):
        logits = model(x)
        loss = criterion(logits.transpose(1, 2), y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
            train_loss, val_loss = evaluate(model, eval_batches)
            print(f'\n\t\t{epoch=} {i=}, {train_loss = :.4f}, {val_loss = :.4f} time elapsed{time.time() - start: .2f}s')
            PATH = f'checkpoints/model_{epoch:02d}_{i:03d}.pt'
            torch.save({'batch': i,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'train_loss': train_loss,
                        'val_loss': val_loss,
                        }, PATH)

  4%|▍         | 5/123 [00:14<05:52,  2.99s/it]


		epoch=0 i=4, train_loss = 3.2865, val_loss = 3.2961 time elapsed 14.73s


  8%|▊         | 10/123 [00:26<05:09,  2.74s/it]


		epoch=0 i=9, train_loss = 3.1351, val_loss = 3.1281 time elapsed 26.30s


 12%|█▏        | 15/123 [00:38<04:54,  2.73s/it]

...

 93%|█████████▎| 115/123 [04:39<00:22,  2.87s/it]


		epoch=4 i=114, train_loss = 1.3057, val_loss = 1.3096 time elapsed 1476.07s


 98%|█████████▊| 120/123 [04:51<00:08,  2.87s/it]


		epoch=4 i=119, train_loss = 1.3047, val_loss = 1.3058 time elapsed 1488.25s


100%|██████████| 123/123 [04:56<00:00,  2.41s/it]


In [ ]:
# !zip -r checkpoints_shakespeare ./checkpoints

In [ ]:
model.eval()
inp = torch.zeros((1,1), dtype=torch.long).to(device)
pred_text = decode(model.generate(inp, 300)[0].tolist())
print(pred_text)



MENENIUS:
Why God's blood,
Not distilighted him; flattering thy
master in the clouds of that love:
If it be warn'd, with mine own duke!

KING RICHARD II:
Speak you brought not so life.

ISABELLA:
Music in your quarrel;
And yet danger to tell thyself, in painted sensible your soul ears,
And I can fi
